In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scraping_class
from matplotlib.dates import DateFormatter
import seaborn as sns
from bs4 import BeautifulSoup

In [2]:
plt.style.use('ggplot')
%matplotlib inline

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
logfile = 'oscar_winners_log.csv'
connector = scraping_class.Connector(logfile)

In [ ]:

movies_data = []

#for i in range(1,600,50):
for i in range(1,100, 100):
#    url_nominees = 'https://www.imdb.com/search/title/?groups=oscar_best_picture_nominees&start=%s&ref_=adv_nxt' % str(i)
    url_winners = 'https://www.imdb.com/search/title/?count=100&groups=oscar_best_picture_winners&sort=year,desc&start=%s&ref_=nv_ch_osc' % str(i)

#    response, call_id = connector.get(url_nominees, 'get_nominees')
    response, call_id = connector.get(url_winners, 'get_winners')
    if response.ok:
        data = response.text
        soup = BeautifulSoup(data, "lxml")
        movies = soup.find_all('div', attrs={'lister-item-content'})


        for movie in movies:
            
            try:
                runtime = movie.find('p', attrs={'text-muted'}).find('span', attrs={'runtime'}).text
                runtime = int(runtime.split(' ')[0])
                
                genre   = movie.find('p', attrs={'text-muted'}).find('span', attrs={'genre'}).text.strip().split(', ')
                    
                
                idx, title, year = movie.find('h3', attrs={'lister-item-header'}).text.strip().split('\n')
                idx = idx.split('.')[0]
                link = movie.find('a')
                link = str(link).split('href="')[1].split('"')[0]
                link  = 'https://www.imdb.com' + link
                
                try:
                    if movie.find_all('span')[-2].text == 'Gross:':
                        gross = movie.find_all('span', attrs={'name':'nv'})[-1].text
                        gross = float(gross.split('$')[1].split('M')[0])
                    else:
                        gross = np.nan
                except:
                    gross = np.nan
                
                try:
                    metascore = movie.find('span', attrs={'metascore'}).text
                except:
                    metascore = np.nan
                    
                job = movie.find_all('p')[2].text
                
                actors = [i.split('\n')[1] for i in job.split('Stars:')[1].split(',')]
                directors = [i.split(',')[0] for i in job.split('Stars:')[0].split('Director')[1].split(':\n')[1].split('\n')[:-2]]
                links_people = [str(i).split('href="')[1].split('"')[0] for i in movies[0].find_all('p')[2].find_all('a')]
                links_people = ["https://www.imdb.com/" + i + "awards" for i in links_people]
                
                try:
                    year = year.split('(I')[1]
                    try:
                        year = year.split('I) (')[1].split(')')[0]
                    except:
                        year = year.split(') (')[1].split(')')[0]
                except:
                    year = year.split('(')[1].split(')')[0]
                    
                movies_data.append([idx, title, year, runtime, genre, metascore, gross, link, directors, actors, links_people])
                
            except:
                NameError
            
    else:
        print('Response failed!')
        
df = pd.DataFrame(movies_data)
df.columns = ['index', 'title', 'year', 'runtime_min', 'genre', 'metascore', 'gross_mil', 'link_movie', 'director', 'actors', 'link_people']

In [ ]:
df

In [271]:
df.to_csv('oscar_nominees.csv')